# Point-to-Mesh Similarity Analysis

This notebook demonstrates how to:
1. Load two meshes
2. Specify a particular point on the source mesh
3. Color-map the target mesh based on similarity to that point

The analysis uses diffusion features + DINO features to compute semantic similarity between mesh vertices.

## 1. Import Dependencies

In [ ]:
import torch
import numpy as np
from dataloaders.mesh_container import MeshContainer
from diffusion import init_pipe
from dino import init_dino
from point_to_mesh_similarity import run_point_similarity_analysis, point_similarity_colormap, visualize_point_similarity
import meshplot as mp
import matplotlib.pyplot as plt

# Set up device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

## 2. Initialize Models

This step loads the diffusion pipeline and DINO model for feature extraction.

In [ ]:
print("Initializing diffusion pipeline...")
pipe = init_pipe(device)

print("Initializing DINO model...")
dino_model = init_dino(device)

print("Models initialized successfully!")

## 3. Load Meshes

Load your source and target meshes. You can modify the paths below to use your own meshes.

In [ ]:
# Mesh paths - modify these to use your own meshes
source_mesh_path = "meshes/cat.off"  # Example mesh
target_mesh_path = "meshes/lion.off"  # Example mesh

print(f"Loading source mesh: {source_mesh_path}")
source_mesh = MeshContainer(source_mesh_path)

print(f"Loading target mesh: {target_mesh_path}")
target_mesh = MeshContainer(target_mesh_path)

print(f"Source mesh: {len(source_mesh.vert)} vertices, {len(source_mesh.face)} faces")
print(f"Target mesh: {len(target_mesh.vert)} vertices, {len(target_mesh.face)} faces")

## 4. Preview Meshes

Let's visualize the meshes before analysis to get familiar with their structure.

In [ ]:
# Preview the meshes side by side
d = mp.subplot(source_mesh.vert, source_mesh.face, s=[2, 2, 0])
mp.subplot(target_mesh.vert, target_mesh.face, s=[2, 2, 1], data=d)
print("Left: Source mesh | Right: Target mesh")

## 5. Configure Analysis Parameters

Set up the parameters for the similarity analysis.

In [ ]:
# Analysis parameters
source_point_idx = 1000  # Choose any vertex index (0 to num_vertices-1)
prompt = "a 3D animal model"  # Text prompt for feature extraction
num_views = 30  # Number of views for rendering (reduced for faster computation)

# Validate the source point index
if source_point_idx >= len(source_mesh.vert):
    source_point_idx = len(source_mesh.vert) // 2  # Use middle vertex as fallback
    print(f"Adjusted source_point_idx to {source_point_idx} (mesh has {len(source_mesh.vert)} vertices)")

print(f"\nAnalysis Configuration:")
print(f"- Source point vertex index: {source_point_idx}")
print(f"- Text prompt: '{prompt}'")
print(f"- Number of views: {num_views}")

## 6. Run Similarity Analysis

This is the main computation step. It will:
1. Extract features for both meshes using multiple camera views
2. Compute similarity between the specified source point and all target vertices
3. Generate the visualization

⚠️ **Note**: This step can take several minutes depending on mesh complexity and number of views.

In [ ]:
print(f"Starting similarity analysis for source vertex {source_point_idx}...")
print("This may take several minutes...\n")

# Run the complete analysis pipeline
similarity_colors, raw_similarities = run_point_similarity_analysis(
    source_mesh=source_mesh,
    target_mesh=target_mesh,
    source_point_idx=source_point_idx,
    device=device,
    pipe=pipe,
    dino_model=dino_model,
    prompt=prompt,
    num_views=num_views
)

print("\n✅ Analysis complete!")

## 7. Analyze Results

Let's examine the similarity scores and find the most similar regions.

In [ ]:
# Find the most similar points on target mesh
top_k = 10
most_similar_indices = np.argsort(raw_similarities)[-top_k:][::-1]
least_similar_indices = np.argsort(raw_similarities)[:top_k]

print(f"📊 Similarity Statistics:")
print(f"- Min similarity: {raw_similarities.min():.4f}")
print(f"- Max similarity: {raw_similarities.max():.4f}")
print(f"- Mean similarity: {raw_similarities.mean():.4f}")
print(f"- Std similarity: {raw_similarities.std():.4f}")

print(f"\n🔥 Top {top_k} most similar vertices on target mesh:")
for i, idx in enumerate(most_similar_indices):
    print(f"  {i+1:2d}. Vertex {idx:5d}: similarity = {raw_similarities[idx]:.4f}")

print(f"\n❄️  Bottom {top_k} least similar vertices on target mesh:")
for i, idx in enumerate(least_similar_indices):
    print(f"  {i+1:2d}. Vertex {idx:5d}: similarity = {raw_similarities[idx]:.4f}")

## 8. Visualize Similarity Distribution

Plot the distribution of similarity scores to understand the data better.

In [ ]:
# Plot similarity distribution
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.hist(raw_similarities, bins=50, alpha=0.7, edgecolor='black')
plt.axvline(raw_similarities.mean(), color='red', linestyle='--', label=f'Mean: {raw_similarities.mean():.3f}')
plt.axvline(raw_similarities.max(), color='green', linestyle='--', label=f'Max: {raw_similarities.max():.3f}')
plt.xlabel('Similarity Score')
plt.ylabel('Frequency')
plt.title('Distribution of Similarity Scores')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(sorted(raw_similarities, reverse=True), linewidth=2)
plt.xlabel('Vertex Rank (sorted by similarity)')
plt.ylabel('Similarity Score')
plt.title('Similarity Scores (Ranked)')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 9. Interactive Visualization

The main result visualization showing:
- **Left**: Source mesh (gray) with the selected point highlighted in red
- **Right**: Target mesh colored by similarity (warmer colors = more similar)

You can interact with the 3D visualization to rotate and zoom the meshes.

In [ ]:
print("🎨 Interactive Visualization:")
print("- Source mesh (left): Gray with red point showing the reference vertex")
print("- Target mesh (right): Colored by similarity (warmer colors = more similar)")
print("- Use mouse to rotate and zoom the 3D view")

# The visualization was already created by run_point_similarity_analysis
# But let's create it again with custom colormap options
visualize_point_similarity(
    source_mesh=source_mesh,
    target_mesh=target_mesh,
    source_point_idx=source_point_idx,
    similarity_colors=similarity_colors,
    colormap='plasma'  # Try 'viridis', 'plasma', 'inferno', 'magma'
)

## 10. Experiment with Different Points

Try analyzing different points on the source mesh to see how similarity patterns change.

In [ ]:
# Experiment with different source points
# You can change these indices and re-run this cell
experiment_points = [100, 500, 1500, 2000]  # Different vertex indices to try

for point_idx in experiment_points:
    if point_idx < len(source_mesh.vert):
        print(f"\n🔬 Experimenting with source vertex {point_idx}...")

        # Quick analysis with fewer views for faster computation
        exp_colors, exp_similarities = point_similarity_colormap(
            device=device,
            pipe=pipe,
            dino_model=dino_model,
            source_mesh=source_mesh,
            target_mesh=target_mesh,
            source_point_idx=point_idx,
            prompt=prompt,
            num_views=10  # Fewer views for quick experiment
        )

        # Show quick stats
        top_match = np.argmax(exp_similarities)
        print(f"  Best match: vertex {top_match} (similarity: {exp_similarities[top_match]:.4f})")

        # Visualize
        visualize_point_similarity(
            source_mesh=source_mesh,
            target_mesh=target_mesh,
            source_point_idx=point_idx,
            similarity_colors=exp_colors,
            colormap='viridis'
        )
    else:
        print(f"Skipping vertex {point_idx} (mesh only has {len(source_mesh.vert)} vertices)")

## 11. Save Results (Optional)

Save the similarity data for later analysis.

In [ ]:
# Save results to files
output_prefix = f"similarity_results_vertex_{source_point_idx}"

# Save similarity scores
np.save(f"{output_prefix}_similarities.npy", raw_similarities)
np.save(f"{output_prefix}_colors.npy", similarity_colors)

# Save analysis summary
summary = {
    'source_mesh_path': source_mesh_path,
    'target_mesh_path': target_mesh_path,
    'source_point_idx': source_point_idx,
    'prompt': prompt,
    'num_views': num_views,
    'min_similarity': float(raw_similarities.min()),
    'max_similarity': float(raw_similarities.max()),
    'mean_similarity': float(raw_similarities.mean()),
    'std_similarity': float(raw_similarities.std()),
    'top_matches': [int(idx) for idx in most_similar_indices[:5]]
}

import json
with open(f"{output_prefix}_summary.json", 'w') as f:
    json.dump(summary, f, indent=2)

print(f"✅ Results saved:")
print(f"  - Similarities: {output_prefix}_similarities.npy")
print(f"  - Colors: {output_prefix}_colors.npy")
print(f"  - Summary: {output_prefix}_summary.json")

## Summary

This notebook demonstrated how to:

1. ✅ Load and visualize 3D meshes
2. ✅ Extract semantic features using diffusion + DINO models
3. ✅ Compute similarity between a specific source point and all target vertices
4. ✅ Visualize results with interactive 3D plots
5. ✅ Analyze similarity distributions and find best matches
6. ✅ Experiment with different source points
7. ✅ Save results for future analysis

### Next Steps

- Try different mesh pairs
- Experiment with different prompts
- Adjust the number of views for quality vs. speed trade-offs
- Use the saved similarity data for further analysis or applications